In [48]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, recall_score

In [49]:
# Load the results table
df = pd.read_csv("ergebnisse.csv")
df.head()

,class,image_id,choice,age,time
0,REAL,000000141807.jpg,REAL,42,3.346280
1,REAL,000000520091.jpg,REAL,42,3.674609
2,AI,image_48.jpg,AI,42,2.114026
3,REAL,000000256447.jpg,REAL,42,3.879275
4,REAL,000000193405.jpg,REAL,42,3.232059


In [50]:
# Add a column to check if choice matches the true class
df['correct'] = df['class'] == df['choice']

In [51]:
# Overall Accuracy
overall_accuracy = df['correct'].mean()
print(f"Overall Accuracy: {overall_accuracy*100:.2f}%")

Overall Accuracy: 80.80%


In [52]:
# More detailed Age Groups
def age_group(age):
    if age < 14:
        return '8-13'
    elif 14 <= age < 20:
        return '14-19'
    elif 20 <= age < 30:
        return '20-29'
    elif 30 <= age < 40:
        return '30-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    else:
        return '60+'

# Assign age groups
df['age_group'] = df['age'].apply(age_group)

In [53]:
# Fix the correct order of age groups
age_order = ['8-13', '14-19', '20-29', '30-39', '40-49', '50-59', '60+']
df['age_group'] = pd.Categorical(df['age_group'], categories=age_order, ordered=True)

# Grouped Accuracy
grouped_accuracy = df.groupby('age_group', observed=True)['correct'].mean().reset_index()

# Grouped Accuracy
grouped_accuracy = df.groupby('age_group', observed=True)['correct'].mean().reset_index()

print("\nAccuracy by Age Group:")
print(grouped_accuracy)



Accuracy by Age Group:
  age_group   correct
0      8-13  0.562500
1     14-19  0.853846
2     20-29  0.866667
3     30-39  0.841176
4     40-49  0.860000
5     50-59  0.792857
6       60+  0.700000


In [54]:
# Precision and Recall by Age Group
precision_recall = []
for group in age_order:
    subset = df[df['age_group'] == group]
    if len(subset) > 0:
        precision = precision_score(subset['class'], subset['choice'], pos_label='AI', average='binary', zero_division=0)
        recall = recall_score(subset['class'], subset['choice'], pos_label='AI', average='binary', zero_division=0)
        accuracy = subset['correct'].mean()
        precision_recall.append({'age_group': group, 'precision': precision, 'recall': recall, 'accuracy': accuracy})

pr_df = pd.DataFrame(precision_recall)

In [55]:
# Plot Precision, Recall, and Accuracy together
fig_combined = px.bar(pr_df, x='age_group', y=['precision', 'recall', 'accuracy'], barmode='group',
                      title='Precision, Recall, and Accuracy by Age Group',
                      labels={'value': 'Score', 'age_group': 'Age Group', 'variable': 'Metric'},
                      text_auto='.2f',
                      color_discrete_sequence=['#FFA15A', '#636EFA', '#AB63FA'],
                      width=1000,
                      height=400)
fig_combined.update_traces(textfont_color='white', textposition='inside')
fig_combined.update_layout(yaxis_range=[0,1])
fig_combined.show()

In [56]:
fig_combined.write_image("stats_by_group.svg")

In [57]:

df['person_id'] = df.index // 10
person_age = df.groupby('person_id', observed=True)['age'].first().reset_index()
person_age['age_group'] = person_age['age'].apply(age_group)
person_age['age_group'] = pd.Categorical(person_age['age_group'], categories=age_order, ordered=True)
person_age_group_counts = person_age['age_group'].value_counts().sort_index()

# Merge grouped_accuracy with person counts
grouped_accuracy['count'] = grouped_accuracy['age_group'].map(person_age_group_counts)

In [58]:
# Plot the distribution of person ages (grouped)
fig2 = px.bar(person_age_group_counts, x=person_age_group_counts.index, y=person_age_group_counts.values,
              labels={'x': 'Age Group', 'y': 'Number of People'},
              title='Distribution of Surveyed People by Age Group',
              text='count',
              width=1000,
              height=400)
fig2.show()


In [59]:
fig2.write_image("age_group_distribution.svg")

In [60]:
labels = ['AI', 'real']
cm = confusion_matrix(df['class'], df['choice'], labels=labels)
fig3 = px.imshow(cm,
                 labels=dict(x="Predicted", y="True", color="Count"),
                 x=labels,
                 y=labels,
                 title="Confusion Matrix",
                 text_auto=True,
                 color_continuous_scale='Blues',
                 width=600,)
fig3.show()

In [61]:
print("\nClassification Report:")
print(classification_report(df['class'], df['choice'], target_names=labels))


Classification Report:
              precision    recall  f1-score   support

          AI       0.79      0.80      0.79       459
        real       0.82      0.82      0.82       541

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000

